# Сессия 5 – Оркестратор для нескольких агентов

Демонстрирует простую двухагентную цепочку (Исследователь -> Редактор) с использованием Foundry Local.


### Объяснение: Установка зависимостей
Устанавливает `foundry-local-sdk` и `openai`, необходимые для локального доступа к модели и завершения чатов. Идемпотентно.


# Сценарий
Реализует минимальный оркестратор с двумя агентами:
- **Агент-исследователь** собирает краткие фактические сведения
- **Агент-редактор** переписывает текст для ясности и удобства восприятия руководством

Демонстрирует использование общей памяти для каждого агента, последовательную передачу промежуточных результатов и простую функцию конвейера. Может быть расширен для дополнительных ролей (например, Критик, Проверяющий) или параллельных ветвей.

**Переменные окружения:**
- `FOUNDRY_LOCAL_ALIAS` - Модель, используемая по умолчанию (по умолчанию: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Основная модель агента (заменяет ALIAS)
- `AGENT_MODEL_EDITOR` - Модель агента-редактора (по умолчанию совпадает с основной)

**Справочник SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Как это работает:**
1. **FoundryLocalManager** автоматически запускает локальную службу Foundry
2. Загружает и активирует указанную модель (или использует кэшированную версию)
3. Предоставляет совместимый с OpenAI интерфейс для взаимодействия
4. Каждый агент может использовать свою модель для выполнения специализированных задач
5. Встроенная логика повторных попыток надежно обрабатывает временные сбои

**Основные возможности:**
- ✅ Автоматическое обнаружение и инициализация службы
- ✅ Управление жизненным циклом модели (загрузка, кэширование, активация)
- ✅ Совместимость с SDK OpenAI для привычного API
- ✅ Поддержка нескольких моделей для специализации агентов
- ✅ Надежная обработка ошибок с логикой повторных попыток
- ✅ Локальное выполнение (без необходимости облачного API)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Объяснение: Основные импорты и типизация
Вводятся dataclasses для хранения сообщений агента и подсказки типов для ясности. Импортируется локальный менеджер Foundry и клиент OpenAI для последующих действий агента.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Объяснение: Инициализация модели (SDK Pattern)
Использует локальный Python SDK Foundry для надежного управления моделями:
- **FoundryLocalManager(alias)** - Автоматически запускает сервис и загружает модель по псевдониму
- **get_model_info(alias)** - Преобразует псевдоним в конкретный ID модели
- **manager.endpoint** - Предоставляет конечную точку сервиса для клиента OpenAI
- **manager.api_key** - Предоставляет API-ключ (опционально для локального использования)
- Поддерживает отдельные модели для разных агентов (основной и редактор)
- Встроенная логика повторных попыток с экспоненциальным увеличением интервала для устойчивости
- Проверка соединения для обеспечения готовности сервиса

**Основной SDK Pattern:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Управление жизненным циклом:**
- Менеджеры хранятся глобально для корректного завершения работы
- Каждый агент может использовать отдельную модель для специализации
- Автоматическое обнаружение сервиса и обработка соединений
- Плавное повторение попыток с экспоненциальным увеличением интервала при сбоях

Это обеспечивает корректную инициализацию перед началом оркестрации агентов.

**Ссылка:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Объяснение: Классы Agent и Memory
Определяет легковесный `AgentMsg` для записей памяти и `Agent`, который включает в себя:
- **Роль системы** - Персонаж агента и его инструкции
- **История сообщений** - Поддерживает контекст беседы
- **Метод act()** - Выполняет действия с надлежащей обработкой ошибок

Агент может использовать разные модели (основную и редакторскую) и поддерживает изолированный контекст для каждого агента. Этот подход обеспечивает:
- Сохранение памяти между действиями
- Гибкое назначение моделей для каждого агента
- Изоляцию и восстановление после ошибок
- Простое связывание и оркестрацию


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Объяснение: Оркестрованный конвейер
Создает двух специализированных агентов:
- **Исследователь**: Использует основную модель, собирает фактическую информацию
- **Редактор**: Может использовать отдельную модель (если настроено), уточняет и переписывает

Функция `pipeline`:
1. Исследователь собирает исходную информацию
2. Редактор преобразует ее в готовый к использованию результат
3. Возвращает как промежуточные, так и окончательные результаты

Этот подход позволяет:
- Специализацию моделей (разные модели для разных ролей)
- Повышение качества через многоэтапную обработку
- Прослеживаемость преобразования информации
- Легкое расширение для большего числа агентов или параллельной обработки


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Объяснение: Выполнение конвейера и результаты
Выполняется конвейер с участием нескольких агентов по вопросу, связанному с соблюдением требований и задержкой, чтобы продемонстрировать:
- Многоэтапное преобразование информации
- Специализацию агентов и их сотрудничество
- Улучшение качества результата через доработку
- Прослеживаемость (сохранение как промежуточных, так и окончательных результатов)

**Структура результата:**
- `question` - Исходный запрос пользователя
- `research` - Сырой результат исследования (фактические пункты)
- `final` - Доработанное резюме для руководства
- `models` - Какие модели использовались на каждом этапе

**Идеи для расширения:**
1. Добавить агента-Критика для проверки качества
2. Реализовать параллельных агентов исследования для разных аспектов
3. Добавить агента-Проверяющего для проверки фактов
4. Использовать разные модели для различных уровней сложности
5. Реализовать циклы обратной связи для итеративного улучшения


### Продвинутое: Настройка пользовательского агента

Попробуйте настроить поведение агента, изменив переменные окружения перед запуском ячейки инициализации:

**Доступные модели:**
- Используйте `foundry model ls` в терминале, чтобы увидеть все доступные модели
- Примеры: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b и т.д.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
